# Distances

In [1]:
from ipynb.fs.full.koselleck import *

[Koselleck] (19:34:16) Alles bereit (+0.0s)


In [2]:
FORCE=False
GBY_LOCAL_O=['corpus1', 'corpus2','period1', 'period2', 'word1', 'word2','qstr']

## Generating distances en masse

In [3]:
def _distvecs(objd): return distvecs(**objd)

def distvecs(period=None,run=None,prefix='dvecs',max_num=10000,num_runs=10,
             num_proc=1,force=False,progress=True,cache_only=False,cache=True):
    
    argd=dict(
        period=period,run=run,prefix=prefix,
        max_num=max_num,num_runs=num_runs,
        num_proc=num_proc,force=force,progress=progress,
        cache_only=cache_only,cache=cache
    )
    odf=None
    
    if period is None:
        # load cached word?
        objs=[{**argd, **{'period':prd, 'progress':True, 'num_proc':1,'cache_only':True}} for prd in get_default_periods()]
        odf=pd.concat(pmap(_distvecs, objs, num_proc=num_proc, desc='Measuring cosine distances across periods', progress=progress))
        return odf if not cache_only else pd.DataFrame()
    
    if run is None:
        objs=[{**argd, **{'run':run+1, 'progress':False, 'num_proc':1,'cache_only':True}} for run in range(num_runs)]
        odf=pd.concat(pmap(_distvecs, objs, num_proc=num_proc, desc='Measuring cosine distances across runs', progress=progress))
        return odf if not cache_only else pd.DataFrame()

    # load vecs?
    dfvecs=vecs(period=period, run=run)
    if not len(dfvecs): return pd.DataFrame()
    
    pprefix=prefix+'_'+period.split('-')[0]#+'_'+str(run).zfill(2)
    ofnfn=os.path.join(PATH_DB,'dvecs',f'data.dvecs.{period}_{run:02}.pkl')
    qstr=f'{period}_{run},max_num={max_num}'
    if cache and not force:
        if os.path.exists(ofnfn):
#             print('Found on disk. Loading...')
            #with open(ofnfn,'rb') as f: od=pickle.load(f)
            odf=read_df(ofnfn)
#             print('Finished loading')
            return odf if not cache_only else pd.DataFrame()
#         with get_veclib(pprefix,autocommit=False) as vl:
#             if qstr in vl:
#                 print('Found in vector library. Loading')
#                 odf=vl[qstr] if not cache_only else pd.DataFrame()
#                 print('Finished loading')
#                 return odf
    # filter
    dfvecs=dfvecs.iloc[:max_num+1]
    
    #distmat!
    if progress: print(f'Computing distances for {max_num} words')
    dfdist=pd.DataFrame(
        fastdist.cosine_pairwise_distance(
            dfvecs.values.astype(float),
            return_matrix=True
        ),
        index=dfvecs.index,
        columns=dfvecs.index
    )
    # csim to cdist?
    dfdist=2-dfdist
    if progress: print('Done')
    
    # convert to long form
#     od = defaultdict(dict)
# #     g=nx.Graph()
#     for w1 in tqdm(dfdist.index,position=0,disable=not progress):
#         for w2,val in dfdist.loc[w1].items():
#             if w1<w2:
# #                 g.add_edge(w1,w2,cdist=val)
#                 od[w1][w2]=val
    if cache:
        dfdist.to_pickle(ofnfn)
        #with open(ofnfn,'wb') as of: pickle.dump(od,of)
#         with get_veclib(pprefix,autocommit=True) as vl:
#             vl[qstr]=od
    return dfdist if not cache_only else pd.DataFrame()

In [4]:
# res=distvecs('1780-1785',1).loc['culture'][['culture','represent']]
# res

In [5]:
# round(distvecs('1780-1785',2).loc['culture'].sort_values(),2)

In [6]:
for prd in get_default_periods():
    distvecs(prd,num_proc=4,cache=True,cache_only=True,force=False)

Measuring cosine distances across runs [x4]: 100%|██████████| 10/10 [00:56<00:00,  5.67s/it]


In [ ]:
prds=get_default_periods()
runs=list(range(1,11))

In [ ]:
# %%timeit
# period=random.choice(prds)
# run=random.choice(runs)
# period,run

In [ ]:
# %%timeit
# distvecs(period,run)#.loc[['culture','virtue']][['culture','virtue']]

## Words

In [8]:

def do_cdist(objd):
    res=cdist(**objd)
    if res is None: res=pd.DataFrame()
    return res

def cdist(word,period=None,run=None,prefix='cdist',neighbors=None,max_num=10000,num_runs=10,num_proc=1,force=False,progress=True,cache_only=False,cache=False):
    index_cols=['word','neighbor','period','run']
    argd=dict(
        word=word,period=period,run=run,prefix=prefix,
        neighbors=neighbors,max_num=max_num,num_runs=num_runs,
        num_proc=num_proc,force=force,progress=progress,
        cache_only=cache_only,cache=cache
    )
    odf=None
    
    if type(word)!=str:
        objs=[{**argd, **{'word':w, 'progress':False, 'num_proc':1}} for w in word]
        odf=pd.concat(pmap(do_cdist, objs, num_proc=num_proc, desc='Measuring cosine distances across words', progress=progress, use_threads=True))
        return odf if not cache_only else pd.DataFrame()
    
    if period is None:
        # load cached word?
        qstr=f'{word},ymin={YMIN},ymax={YMAX},ybin={YEARBIN}'
        if cache:
            with get_veclib('cdist',autocommit=False) as vl:
                if qstr in vl: return vl[qstr]
        
        objs=[{**argd, **{'period':prd, 'progress':False, 'num_proc':num_proc}} for prd in get_default_periods()]
        odf=pd.concat(pmap(do_cdist, objs, num_proc=1, desc='Measuring cosine distances across periods', progress=progress, use_threads=True))
        if cache:
            with get_veclib('cdist',autocommit=True) as vl:
                vl[qstr]=odf
        return odf if not cache_only else pd.DataFrame()
    
    if run is None:
        qstr=f'{word}_{period}'
        if not force and cache:
            with get_veclib(prefix) as vl: odf=vl.get(qstr)
            #odf=vl.get(qstr)
        if odf is None:
            objs=[{**argd, **{'run':run+1, 'progress':False, 'num_proc':1}} for run in range(num_runs)]
            odf=pd.concat(pmap(do_cdist, objs, num_proc=num_proc, desc='Measuring cosine distances across runs', progress=progress, use_threads=True))
            if cache: #vl[qstr]=odf
                with get_veclib(prefix,autocommit=True) as vl:
                    vl[qstr]=odf
        return odf if not cache_only else pd.DataFrame()
            

    # load vecs?
    dfdist=distvecs(period,run,cache=True)
    distwords=set(dfdist.index)
    if not word in distwords: return pd.DataFrame()
    
    sdist=dfdist.loc[word]
    sdist=sdist[distwords-{word}]
    sdist=sdist*-1 if sdist.min()<-0.1 else sdist
    if neighbors: sdist=sdist[(set(neighbors) & set(sdist.index))]
    wddf=pd.DataFrame()
    wddf['cdist']=sdist
    wddf=wddf.rename_axis('neighbor').sort_values('cdist')
    wddf['word']=word
    wddf['period']=period
    wddf['run']=run
    wddf=wddf.reset_index().set_index(index_cols)
    return wddf


In [12]:
# %%timeit
# cdist('culture','1760-1765',1)

In [ ]:
dists=cdist('culture',neighbors=['plant','soil','education'],num_proc=1)
dists

In [ ]:
distvecs('1770-1775',1)